# CPW media example

In [ ]:
%load_ext autoreload
%autoreload 2
import skrf as rf
from skrf.calibration import IEEEP370_SE_NZC_2xThru
from skrf.media import CPW
import numpy as np
import matplotlib.pyplot as plt

rf.stylely()

## Measurement of two CPWG lines with different lengths

The measurement where performed the 21th March 2017 on a Anritsu MS46524B 20GHz Vector Network Analyser. The setup is a linear frequency sweep from 1MHz to 10GHz with 10'000 points. Output power is 0dBm, IF bandwidth is 1kHz and neither averaging nor smoothing are used.

CPWGxxx is a L long, W wide, with a G wide gap to top ground, T thick copper coplanar waveguide on ground on a H height substrate with top and bottom ground plane. A closely spaced via wall is placed on both side of the line and the top and bottom ground planes are connected by many vias.

| Name | L (mm) | W (mm) | G (mm) | H (mm) | T (um) | Substrate |
| :--- | ---: | ---: | ---: | ---: | ---: | :--- |
| CPW100 | 100 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |
| CPW200 | 200 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |

The milling of the artwork is performed mechanically with a lateral wall of 45°.

The relative permittivity of the dielectric was assumed to be approximately 4.5 for design purpose.

![MSL100 and MSL200 illustration, both are microstripline, MSL200 is twice the length of MSL100](MSL_CPWG_100_200.jpg "MSL100 and MSL200")

## Removing connectors effects

The SMA connectors cause a discontinuity in the signal path. The waves have to go trough a coaxial to planar transition with a complex three dimensionnal geometry in the launch region.

By deembedding the smaller length line from the longer line, the connector and a quarter of line will be removed on both sides.

In [ ]:
# Load raw measurements
TL100 = rf.Network('CPWG100.s2p')
TL200 = rf.Network('CPWG200.s2p')
TL100_dc = TL100.extrapolate_to_dc(kind='linear')
TL200_dc = TL200.extrapolate_to_dc(kind='linear')

# deembedding
dm = IEEEP370_SE_NZC_2xThru(dummy_2xthru = TL100, name = '2xthru')
side1 = dm.s_side1
side1.name = 'side1'
side2 = dm.s_side2
side2.name = 'side2'
d_dut = dm.deembed(TL200)
d_dut.name = 'd_dut'
side1_dc = side1.extrapolate_to_dc(kind='cubic')
side2_dc = side2.extrapolate_to_dc(kind='cubic')
d_dut_dc = d_dut.extrapolate_to_dc(kind='cubic')

# plot them all
plt.figure()
plt.suptitle('Time domain reflexion step response (DC extrapolation)')
TL100_dc.s11.plot_z_time_step()
TL200_dc.s11.plot_z_time_step()
side1_dc.s11.plot_z_time_step()
side2_dc.s22.plot_z_time_step()
d_dut_dc.s11.plot_z_time_step()
plt.xlim(-2, 4)
plt.show()

## Check deembedding
Looks at the residuals by deembedding two sides models from 2xthru. Residuals insertion magnitude shall be ± 0.1 dB and residual insertion loss phase shall be within ± 1°.

This is the case for insertion loss magnitude but the phase exhibit a curious 180° step portion.

In [ ]:
# compute residuals
res = side1.inv ** TL100 ** side2.inv
res.name = 'residuals'
res.s += 1e-15 # avoid numeric singularities

# plot them all
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
res.plot_s_db(1,0)
plt.subplot(1, 2, 2)
res.plot_s_deg(1,0)

## Comparison of dembedded with same length line with connectors
Deembedding of connectors and lanch remove frequency rising slope in return loss and some length and ringing in insertion loss. 

In [ ]:
# plot them all
plt.figure(figsize=(10, 5))
plt.subplot(2, 2, 1)
d_dut.plot_s_db(0,0)
TL100.plot_s_db(0,0)
plt.subplot(2, 2, 2)
d_dut.plot_s_deg(0,0)
TL100.plot_s_deg(0,0)
plt.subplot(2, 2, 3)
d_dut.plot_s_db(1,0)
TL100.plot_s_db(1,0)
plt.subplot(2, 2, 4)
d_dut.plot_s_deg(1,0)
TL100.plot_s_deg(1,0)

## Comparison with media
Effect of metallization thickness is neglected.

In [ ]:
ep_r = 4.5
tanD = 0.018

cpw = CPW(frequency=d_dut.frequency, w = 1.7e-3, s = 0.5e-3, t = None, h = 1.55e-3,
        ep_r = ep_r, tand = tanD, rho = 1.7e-8, Z0 = 50, has_metal_backside = True,
         compatibility_mode = 'ads')
l = cpw.line(d = 100.0e-3, unit = 'm', embed = True, z0 = cpw.Z0)
l.name = 'CPW ads'

cpw2 = CPW(frequency=d_dut.frequency, w = 5.6e-3, s = 0.5e-3, t = None, h = 1.55e-3,
        ep_r = ep_r, tand = tanD, rho = 1.7e-8, Z0 = 50, has_metal_backside = True,
        compatibility_mode = 'qucs')
l2 = cpw2.line(d = 100.0e-3, unit = 'm', embed = True, z0 = cpw2.Z0)
l2.name = 'CPW qucs'

# plot them all
plt.figure(figsize=(10, 10))
plt.subplot(2,2,1)
d_dut.plot_s_db(0,0)
l.plot_s_db(0,0, marker = '.', linestyle = 'none', markevery = 10)
l2.plot_s_db(0,0, marker = 'x', linestyle = 'none', markevery = 10)
plt.subplot(2,2,2)
d_dut.plot_s_deg(0,0)
l.plot_s_deg(0,0, marker = '.', linestyle = 'none', markevery = 100)
l2.plot_s_deg(0,0, marker = '.', linestyle = 'none', markevery = 100)
plt.subplot(2,2,3)
d_dut.plot_s_db(1,0)
l.plot_s_db(1,0, marker = '.', linestyle = 'none', markevery = 200)
l2.plot_s_db(1,0, marker = 'x', linestyle = 'none', markevery = 200)
plt.subplot(2,2,4)
d_dut.plot_s_deg(1,0)
l.plot_s_deg(1,0, marker = '.', linestyle = 'none', markevery = 100)
l2.plot_s_deg(1,0, marker = '.', linestyle = 'none', markevery = 100)
